In [1]:
# Benchmark models
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from preprocessing import complete_vars
from preprocessing import ratios
from preprocessing import breakdown_vars
from preprocessing import dummies_ohe
from preprocessing import Xy
from preprocessing import std_z
from imblearn.under_sampling import RandomUnderSampler


from Genfet import GaFeatureSelection
from Genfet import Geafet
from gentools import historical_fittest
from gentools import PerformanceTest
from gentools import get_names
from gentools import plot_stats
from ineq import Theil
from sklearn.linear_model import LogisticRegression



# Models
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.metrics import f1_score  #ytrue, ypred
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

from sklearn.metrics import classification_report


from GeneticNeuralNetwork import gaMLP_Entropy
from sklearn.model_selection import StratifiedKFold
from ineq import Theil
from gentools import plot_stats
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

from LocalSearch import ModelArchitecture

# Third libraries
import pandas as pd
from tensorflow.keras import Sequential
import tensorflow as tf
from NeuralNetwork import  MLP
import itertools
from Experiment import hypersearch_GeneticNN
from Experiment import eval_model_NN   
from Experiment import mean_results_NN

In [2]:
df_train = pd.read_csv("Datapooled.csv")

VARS = ['Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 'Patrimonio total',
     'Total pasivos', 'Total de activos', 'Ganancias acumuladas',  'Pasivos corrientes totales',  'Activos corrientes totales']

df_train.rename(columns={'Clasificación Industrial Internacional Uniforme Versión 4 A.C':'Sector'}, inplace=True)

df_train = df_train[VARS+[ 'event', 'Sector']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information 
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'GPM', 'NPM', 'ROE','ROA', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector']
print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.drop(columns=['complete-vars'], inplace=True)
df_train = df_train[predictors + ['event']]

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Ganancia bruta                      353 non-null    float64
 1   Ganancia (pérdida)                  353 non-null    float64
 2   Ingresos de actividades ordinarias  353 non-null    float64
 3   Costo de ventas                     330 non-null    float64
 4   Patrimonio total                    353 non-null    float64
 5   Total pasivos                       353 non-null    float64
 6   Total de activos                    353 non-null    float64
 7   Ganancias acumuladas                351 non-null    float64
 8   Pasivos corrientes totales          352 non-null    float64
 9   Activos corrientes totales          353 non-null    float64
 10  event                               771 non-null    float64
 11  Sector                              771 non-null

In [3]:
X, y = Xy(df_train, 'event')
cat, nonormal, normal  = breakdown_vars(X)
nums = nonormal + normal
X = dummies_ohe(X, cat)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)
rus = RandomUnderSampler(random_state=123)
X_train, y_train = rus.fit_resample(X_train, y_train)
X_train = std_z(nums, X_train)
X_test = std_z(nums = nums, df_ = X_test)

vars = ['ROE', 'ROA',
         'IR', 'RSL', 
         'Sector_D', 'Sector_E', 
         'Sector_G','Sector_J', 
         'Sector_K', 'Sector_L', 
         'Sector_M', 'Sector_O', 
         'Sector_Q','Sector_R']


X_train, X_test = X_train.loc[:, vars],  X_test.loc[:, vars]

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [4]:
y_train.value_counts()

event
0.0    254
1.0    254
Name: count, dtype: int64

In [5]:
y_test.value_counts()

event
0.0    3296
1.0      64
Name: count, dtype: int64

In [6]:
# Logistic Regression
def grid_lr(X_train, y_train):
    model = LogisticRegression(random_state=666, max_iter=1500)
    solvers = ['liblinear']
    penalty = ['l2','l1',]
    c_values = [1000, 100, 10, 1.0, 0.1, 0.01, 0.001,0.0001 , 0.00001 ]
    grid = dict(solver=solvers,penalty=penalty,C=c_values)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='f1',error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return  grid_result.best_estimator_



"""
def grid_RandomForest(X_train, y_train):
  model = RandomForestClassifier(random_state=666)
  n_estimators =  [100, 300, 500, 800]
  criterion = ['gini', 'entropy', 'log_loss']
  max_depth  =  [None, 5, 10, 30]
  min_samples_split =  [2, 5, 10, 15]
  min_samples_leaf  =[1, 2, 4, 7]
  max_features = ['sqrt', 'log2']


  grid = dict(n_estimators = n_estimators, criterion = criterion,  
              min_samples_split = min_samples_split,  
              max_features=max_features,
              max_depth = max_depth,
              min_samples_leaf = min_samples_leaf
              )
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                            scoring='f1',error_score='raise')
  grid_result = grid_search.fit(X_train, y_train)
  return  grid_result.best_estimator_
"""



# Support Vector Machine
def grid_SVM(X_train, y_train, performance_metric='f1', resultsGrid=False):
    model = SVC(random_state=666)
    C = np.linspace(0.000001 , 100, 100)
    kernels = ['poly', 'rbf', 'linear']
    gamma = ['scale', 'auto']
    grid = dict(C = C, kernel = kernels, gamma = gamma)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring=performance_metric,error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    if resultsGrid==True:
        return grid_result.cv_results_
    else:
        return  grid_result.best_estimator_

In [7]:
model_lr = grid_lr(X_train, y_train)
lr_predict = model_lr.predict(X_test)
print(classification_report(y_test, lr_predict))
lr_table  = pd.DataFrame(classification_report(y_test, lr_predict, output_dict=True)).iloc[:,0:2]


              precision    recall  f1-score   support

         0.0       1.00      0.48      0.65      3296
         1.0       0.04      0.97      0.07        64

    accuracy                           0.49      3360
   macro avg       0.52      0.73      0.36      3360
weighted avg       0.98      0.49      0.64      3360



In [8]:
"""
model_random_forest  = grid_RandomForest(X_train, y_train)
random_forest_predict = model_random_forest.predict(X_test)
print(classification_report(y_test, random_forest_predict))
random_forest_table  = pd.DataFrame(classification_report(y_test,  random_forest_predict, output_dict=True)).iloc[:,0:2]
"""

'\nmodel_random_forest  = grid_RandomForest(X_train, y_train)\nrandom_forest_predict = model_random_forest.predict(X_test)\nprint(classification_report(y_test, random_forest_predict))\nrandom_forest_table  = pd.DataFrame(classification_report(y_test,  random_forest_predict, output_dict=True)).iloc[:,0:2]\n'

In [9]:
model_SVM  = grid_SVM(X_train, y_train)
SVM_predict = model_SVM.predict(X_test)
print(classification_report(y_test, SVM_predict))
SVM_table  = pd.DataFrame(classification_report(y_test,  SVM_predict, output_dict=True)).iloc[:,0:2]


              precision    recall  f1-score   support

         0.0       1.00      0.47      0.64      3296
         1.0       0.03      0.95      0.07        64

    accuracy                           0.48      3360
   macro avg       0.52      0.71      0.35      3360
weighted avg       0.98      0.48      0.63      3360



In [10]:
from sklearn.neural_network import MLPClassifier

def grid_MLP(X_train, y_train):
  model = MLPClassifier(random_state=123)
  hidden_layer_sizes =  [(5, 5)]
  activation = ['logistic']
  solver =  ['sgd'] 
  learning_rate = ['constant', 'invscaling', 'adaptive']
  alpha   =  [0.00001, 0.0001, 0.001, 0.01, 1]
  learning_rate_init = [0.00001, 0.0001, 0.001, 0.01, 1]
  batch_size = [X_train.shape[0]]
  momentum = [0.5, 0.8,  0.9 , 1]
  max_iter = [500, 700, 1000, 1500, 2000]
  grid = dict(hidden_layer_sizes = hidden_layer_sizes,
              solver = solver,
              alpha = alpha,
              max_iter = max_iter,
              activation = activation,
              batch_size = batch_size,
              learning_rate_init = learning_rate_init,
              momentum = momentum,
              learning_rate = learning_rate)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                            scoring='f1',error_score='raise')
  grid_result = grid_search.fit(X_train, y_train)
  return  grid_result.best_estimator_

In [11]:
model_mlp  = grid_MLP(X_train, y_train)
NN_predict = model_mlp.predict(X_test)
print(classification_report(y_test, NN_predict))
MLP_table  = pd.DataFrame(classification_report(y_test,  NN_predict, output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

         0.0       1.00      0.43      0.60      3296
         1.0       0.03      0.98      0.06        64

    accuracy                           0.44      3360
   macro avg       0.52      0.71      0.33      3360
weighted avg       0.98      0.44      0.59      3360



In [12]:
models_tab = pd.concat([lr_table, SVM_table, MLP_table], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic regression','No-Default'),("Logistic regression",'Deafult'),
              ("Support vector machine",'No-Default'),('Support vector machine','Default'),
              ('Backpropagation NN', 'No-default'), ('Backpropagation NN', 'Default'),])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("benchmark-models.tex")
models_tab

In [13]:
####################################
# Epochs determination in NN...

from LocalSearch import ModelArchitecture
from tensorflow.keras import Sequential
from Experiment import mean_results_NN


architecture = (5,5,1)


errors_mean  = []
model = Sequential()
model  = ModelArchitecture(model, architecture, X_train)
model.compile(optimizer="SGD",
              loss='binary_crossentropy',
              metrics=['accuracy'])
EPOCHS  = [2, 5]
for epoch in EPOCHS:
  results = eval_model_NN(classifier  = model,
              X_train_ =  X_train,
              y_train_  = y_train,
              epochs =epoch,
              k_folds= 5)
  errors_mean.append(mean_results_NN(results, 'f1'))
  Optimun_epochs  = EPOCHS[np.array(errors_mean).argmax()]


model = Sequential()
model  = ModelArchitecture(model, architecture, X_train)
model.compile(optimizer="SGD",
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=Optimun_epochs, verbose=1)
preds_backproptf = np.where(model.predict(X_test)>0.5,1,0)



Epoch 1/2


13/13 [==============================] - 1s 6ms/step - loss: 0.8269 - accuracy: 0.5000
Epoch 2/2
4/4 [==============================] - 0s 2ms/step
Epoch 1/2
13/13 [==============================] - 0s 2ms/step - loss: 0.7911 - accuracy: 0.5000
Epoch 2/2
13/13 [==============================] - 0s 2ms/step - loss: 0.7770 - accuracy: 0.5000


c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


4/4 [==============================] - 0s 2ms/step
Epoch 1/2
13/13 [==============================] - 0s 2ms/step - loss: 0.7648 - accuracy: 0.5000
Epoch 2/2
1/4 [======>.......................] - ETA: 0s

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


4/4 [==============================] - 0s 3ms/step
Epoch 1/2
13/13 [==============================] - 0s 2ms/step - loss: 0.7458 - accuracy: 0.4988
Epoch 2/2
13/13 [==============================] - 0s 2ms/step - loss: 0.7382 - accuracy: 0.4988


...

Epoch 4/5
13/13 [==============================] - 0s 4ms/step - loss: 0.6998 - accuracy: 0.5000
Epoch 5/5
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6978 - accuracy: 0.5000
Epoch 2/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6970 - accuracy: 0.5000
Epoch 3/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6963 - accuracy: 0.5000


c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 4/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6957 - accuracy: 0.5000
Epoch 5/5
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6949 - accuracy: 0.5012
Epoch 2/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6945 - accuracy: 0.5037
Epoch 3/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6942 - accuracy: 0.5037
Epoch 4/5
13/13 [==============================] - 0s 6ms/step - loss: 0.6940 - accuracy: 0.5037
Epoch 5/5
4/4 [==============================] - 0s 1ms/step
Epoch 1/5
13/13 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.5061
Epoch 2/5
 1/13 [=>............................] - ETA: 0s - loss: 0.7063 - accuracy: 0.3438

c:\Users\sjrp2\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


13/13 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5086
Epoch 3/5
13/13 [==============================] - 0s 3ms/step - loss: 0.6929 - accuracy: 0.5086
Epoch 4/5
13/13 [==============================] - 0s 3ms/step - loss: 0.6928 - accuracy: 0.5086
Epoch 5/5
4/4 [==============================] - 0s 2ms/step
Epoch 1/5
16/16 [==============================] - 0s 2ms/step - loss: 0.7044 - accuracy: 0.5000
Epoch 2/5
16/16 [==============================] - 0s 2ms/step - loss: 0.7025 - accuracy: 0.5000
Epoch 3/5
16/16 [==============================] - 0s 2ms/step - loss: 0.7009 - accuracy: 0.5000
Epoch 4/5
16/16 [==============================] - 0s 2ms/step - loss: 0.6995 - accuracy: 0.5000
Epoch 5/5
105/105 [==============================] - 0s 1ms/step


In [14]:
X_train.describe()

,ROE,ROA,IR,RSL,Sector_D,Sector_E,Sector_G,Sector_J,Sector_K,Sector_L,Sector_M,Sector_O,Sector_Q,Sector_R
count,508.000000,5.080000e+02,5.080000e+02,5.080000e+02,508.0,508.000000,508.000000,508.000000,508.000000,508.000000,508.000000,508.0,508.000000,508.000000
mean,0.000000,2.797412e-17,1.398706e-16,1.678447e-16,0.0,0.001969,0.297244,0.031496,0.019685,0.055118,0.053150,0.0,0.003937,0.007874
std,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,0.0,0.044368,0.457495,0.174826,0.139053,0.228436,0.224553,0.0,0.062684,0.088473
min,-20.432386,-1.442665e+01,-9.037867e-01,-1.877211e+00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.031782,1.892854e-02,-3.347233e-01,-9.058330e-01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,0.053442,1.455670e-01,-7.805332e-02,1.117814e-01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,0.090878,2.686279e-01,1.400104e-01,1.032800e+00,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
max,3.526417,1.761250e+00,1.766741e+01,1.169591e+00,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000


In [15]:
X_train.index

Index([11250, 14336,  9571,  3175,  8136,  3385,   822, 12247,  3985, 13209,
       ...
        4788,  4985,  2971,  6943,  9109,  1157,  3242,  9792,  9255,   987],
      dtype='int64', length=508)

In [16]:
y_test.index

Index([19146,  3820, 10870, 17643, 18621,  9242, 13245, 20033,   952,  4099,
       ...
        1161, 19710,  9821, 15427,  8818, 10170,  8837,  3652, 19586, 13580],
      dtype='int64', length=3360)